# Notebook de modelisation

### Chris Hozé

#### On utilise tensorflow donc on se place dans l'environnement tensor_env paramétré avec tensor_gpu


In [4]:
# Import des 

# Import des packages 
import pandas as pd
import numpy as np
import pathlib
import os
#from scipy import sparse

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

import cv2

# Keras et tensorflow
import tensorflow as tf

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D 
from tensorflow.keras.layers import MaxPooling2D

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

# # sklearn
# from sklearn.decomposition import PCA
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import KFold, train_test_split   
# from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay


In [2]:
# On vérifie que la gpu fonctionne

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# En repartant du dataset tabulaire : c'est possible mais pas ideal car il est long à créer
Cela pourrait tout de même être intéressant pour pouvoir utiliser l info URL

In [ ]:
# Charger le dataset tabulaire préprocessé ne fonctionne pas pour utilisation de tensorflow

# file = r"C:\Users\Inrae\Documents\Projet_Data_Science\radio_tab.csv"

# # Dataset complet mais reduit en 100*100

# df = pd.read_csv(file)
# # On cree un jeu de données réduit en respectant les proportions initiales 

# df.rename(columns = {'Unnamed: 0':'FILENAME'}, inplace = True)

# df = df.set_index("FILENAME")

# #df = df.rename(columns = {'Unnamed : 0' : 'FILENAME'})
# df.head()


# On recode la variable source

#cat_url = pd.get_dummies(df["URL"], prefix="url")

#df = pd.concat([cat_url, df], axis=1)

#X = df.drop(["URL","SIZE","FORMAT","TYPE","num"], axis=1)

#X.head()


# En repartant du notebook de MME pour l'import des données

In [3]:
# On recrée un répertoire de données préprocessées, redimensionnées en 224*224 avec filtre gaussien et masquage

path_to_data = r"C:\Users\Inrae\Documents\Projet_Data_Science"
data_folder_path = os.path.join(path_to_data,"COVID-19_Radiography_Dataset")
output_path = os.path.join(path_to_data,"processed")
folder_to_process = ["Lung_Opacity","COVID","Normal","Viral_Pneumonia"]

make_tiny=False
tiny_size=1000
final_size=(224,224)

for img_type in  folder_to_process:
    print(f"Processing folder: {img_type}")

    img_folder_path = os.path.join(data_folder_path,img_type,"images")
    mask_folder_path = os.path.join(data_folder_path,img_type,"masks")

    output_folder_path = os.path.join(output_path, img_type)
    #output_folder_path.mkdir(parents=True, exist_ok=True)

    nb_image_done = 0
    for image_name, mask_name in zip(os.listdir(img_folder_path),
                                     os.listdir(mask_folder_path)):

        image_path = os.path.join(img_folder_path, image_name)
        mask_path = os.path.join(mask_folder_path, mask_name)

        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # resized to mask size
        image = cv2.resize(image, dsize = (256, 256)) # fixed size to mask size

        # masking
        res =  cv2.bitwise_and(image, image, mask=mask)
        res = cv2.GaussianBlur(res, ksize = (3,3), sigmaX=0.2)
        res = cv2.resize(res, dsize = final_size)

        # Write masked image
        output_image_name = image_name + '_masked.png'
        output_file =  os.path.join(output_folder_path, output_image_name)
        cv2.imwrite(output_file, res)

        nb_image_done += 1
        if make_tiny and nb_image_done >= tiny_size:
            break

    print(f"Processing folder: {img_type} done.")


Processing folder: Lung_Opacity


NameError: name 'cv2' is not defined

In [5]:

path_to_data = r"C:\Users\Inrae\Documents\Projet_Data_Science"
data_folder_path = os.path.join(path_to_data,"COVID-19_Radiography_Dataset")
output_path = os.path.join(path_to_data,"processed")
final_size=(224,224)

# Import des données on sépare entre apprentissage et validation
batch_size=32
data_dir=output_path
size=final_size

# On laisse en couleur pour pouvoir utiliser les modeles preentrainés

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.5,
    subset="training",
    #color_mode= "grayscale",
    seed=42,
    image_size=size,
    batch_size=batch_size)


val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.5,
    subset="validation",
    #color_mode= "grayscale",
    seed=42,
    image_size=size,
    batch_size=batch_size)



Found 21165 files belonging to 4 classes.
Using 10583 files for training.
Found 21165 files belonging to 4 classes.
Using 10582 files for validation.


In [6]:
# Création X_train, X_test, y_train, y_test

# For train data
all_images = []
all_labels = []

for images, labels in train_ds.take(-1):  # -1 takes all
    all_images.append(images.numpy())
    all_labels.append(labels.numpy())

X_train = np.concatenate(all_images)
y_train = np.concatenate(all_labels)

# For test data
all_images = []
all_labels = []

for images, labels in val_ds.take(-1):  # -1 takes all
    all_images.append(images.numpy())
    all_labels.append(labels.numpy())

X_test = np.concatenate(all_images)
y_test = np.concatenate(all_labels)

# normalization
X_train = X_train / 255
X_test = X_test / 255

# encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [7]:
# On cree un generateur d image

train_datagen = ImageDataGenerator(
    shear_range = 0.1, # random application of shearing
    zoom_range = 0.1,
    #horizontal_flip = False,
    brightness_range = (0.4, 0.6),
    #width_shift_range=0.1,
    #height_shift_range=0.1  
    ) 

test_datagen = ImageDataGenerator()

# Augmenter respectivement les jeu de données d'entrainement
train_dataset = train_datagen.flow(X_train,y_train,   batch_size = 64)

test_dataset = test_datagen.flow(X_test, y_test, batch_size = 64)



# Modèle de DL
## Test d un premier modèle from scratch

In [14]:
# Construction d un modèle classique

model = Sequential()

# Couche d'entrée pour les images 224x224 avec 3 canaux (RGB) 
model.add(Input(shape=(224, 224, 3))) 

# Première couche de convolution 
model.add(Conv2D(16, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D((2, 2))) 

# Deuxième couche de convolution 
model.add(Conv2D(32, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D((2, 2))) 

# Troisième couche de convolution 
model.add(Conv2D(64, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D((2, 2)))

# Quatireme couche de convolution 
model.add(Conv2D(128, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D((2, 2)))


# Couche flatten pour transformer les cartes de caractéristiques en un vecteur 
model.add(Flatten()) 

# Ajouter des couches fully connected (denses) 

model.add(Dense(60, activation='relu')) 
model.add(Dense(4, activation='softmax')) 

#Compiler le modèle model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Définir le learning rate désiré 
learning_rate = 0.001 #

#Créer une instance de l'optimiseur Adam avec le learning rate personnalisé  
optimizer = Adam(learning_rate=learning_rate) 

# Compiler le modèle avec l'optimiseur personnalisé 
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])

#Entrainement du modèle, utiliser le jeu de données augmenté, et préciser les callbacks 

batch_size = 64
epochs = 10 
model_history = model.fit(
            train_dataset, # use augmented images for train 
            steps_per_epoch=X_train.shape[0] // batch_size,
            validation_data = test_dataset, # use augmented images for test
            epochs = epochs,
            verbose=True)

Epoch 1/10
165/165 [==============================] - 81s 484ms/step - loss: 1.8190 - accuracy: 0.6308 - val_loss: 1.3669 - val_accuracy: 0.4838
Epoch 2/10
165/165 [==============================] - 81s 493ms/step - loss: 0.6717 - accuracy: 0.7338 - val_loss: 1.3680 - val_accuracy: 0.4838
Epoch 3/10
165/165 [==============================] - 85s 516ms/step - loss: 0.6413 - accuracy: 0.7498 - val_loss: 1.3635 - val_accuracy: 0.4838
Epoch 4/10
165/165 [==============================] - 86s 519ms/step - loss: 0.6013 - accuracy: 0.7693 - val_loss: 1.3633 - val_accuracy: 0.4838
Epoch 5/10
165/165 [==============================] - 84s 508ms/step - loss: 0.5782 - accuracy: 0.7716 - val_loss: 1.3617 - val_accuracy: 0.4838
Epoch 6/10
165/165 [==============================] - 85s 516ms/step - loss: 0.5632 - accuracy: 0.7790 - val_loss: 1.3595 - val_accuracy: 0.4838
Epoch 7/10
165/165 [==============================] - 92s 556ms/step - loss: 0.5378 - accuracy: 0.7902 - val_loss: 1.3574 - val_ac

In [10]:
# On importe un modele préentrainé : REsnet50
# https://keras.io/api/applications/#finetune-inceptionv3-on-a-new-set-of-classes
#import h5py

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

base_model = ResNet50(weights='imagenet')

base_model.summary()





Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [11]:

# add a global spatial average pooling layer
x = base_model.output

#x = MaxPooling4D(4,4)(x)

# Couche flatten pour transformer les cartes de caractéristiques en un vecteur 
x = Flatten()(x)

# Ajouter des couches fully connected (denses) 

x = Dense(60, activation='relu')(x) 

predictions = Dense(4, activation='softmax')(x) 

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first:  freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='Adam', loss='categorical_crossentropy',   metrics = ["accuracy"])

# train the model on the new data for a few epochs
model.fit(train_dataset, # use augmented images for train 
            steps_per_epoch=X_train.shape[0] // batch_size,
            validation_data = test_dataset, # use augmented images for test
            epochs = 10,
            verbose=True)



Epoch 1/10
165/165 [==============================] - 132s 778ms/step - loss: 1.1584 - accuracy: 0.5164 - val_loss: 1.3760 - val_accuracy: 0.2801
Epoch 2/10
165/165 [==============================] - 139s 840ms/step - loss: 0.9659 - accuracy: 0.6235 - val_loss: 1.9295 - val_accuracy: 0.2801
Epoch 3/10
165/165 [==============================] - 142s 858ms/step - loss: 0.8832 - accuracy: 0.6486 - val_loss: 2.3660 - val_accuracy: 0.2801
Epoch 4/10
165/165 [==============================] - 142s 863ms/step - loss: 0.8462 - accuracy: 0.6653 - val_loss: 2.5808 - val_accuracy: 0.2801
Epoch 5/10
165/165 [==============================] - 147s 889ms/step - loss: 0.8135 - accuracy: 0.6805 - val_loss: 2.7803 - val_accuracy: 0.2801
Epoch 6/10
165/165 [==============================] - 149s 902ms/step - loss: 0.7964 - accuracy: 0.6886 - val_loss: 2.8758 - val_accuracy: 0.2801
Epoch 7/10
165/165 [==============================] - 139s 843ms/step - loss: 0.7870 - accuracy: 0.6936 - val_loss: 2.9264 -

On degele les parametres des 10 dernieres couches

In [15]:
# ## Vu la precision on unfreeze certaine couche

# add a global spatial average pooling layer
x = base_model.output

#x = MaxPooling4D(4,4)(x)

# Couche flatten pour transformer les cartes de caractéristiques en un vecteur 
x = Flatten()(x)

# Ajouter des couches fully connected (denses) 

x = Dense(60, activation='relu')(x) 

predictions = Dense(4, activation='softmax')(x) 


# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:165]:
   layer.trainable = False
for layer in model.layers[165:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
#from tenkeras.optimizers import SGD
#model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

#model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=["accuracy"])

# Définir le learning rate désiré 
learning_rate = 0.0001 #

#Créer une instance de l'optimiseur Adam avec le learning rate personnalisé  
optimizer = Adam(learning_rate=learning_rate) 

# Compiler le modèle avec l'optimiseur personnalisé 
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])

# train the model on the new data for a few epochs
model.fit(train_dataset, # use augmented images for train 
            steps_per_epoch=X_train.shape[0] // batch_size,
            validation_data = test_dataset, # use augmented images for test
            epochs = 10,
            verbose=True)


Epoch 1/10
165/165 [==============================] - 80s 481ms/step - loss: 0.4531 - accuracy: 0.8238 - val_loss: 1.3551 - val_accuracy: 0.2765
Epoch 2/10
165/165 [==============================] - 80s 485ms/step - loss: 0.4581 - accuracy: 0.8179 - val_loss: 1.3551 - val_accuracy: 0.2765
Epoch 3/10
165/165 [==============================] - 88s 532ms/step - loss: 0.4599 - accuracy: 0.8196 - val_loss: 1.3551 - val_accuracy: 0.2765
Epoch 4/10
165/165 [==============================] - 83s 503ms/step - loss: 0.4587 - accuracy: 0.8194 - val_loss: 1.3551 - val_accuracy: 0.2765
Epoch 5/10
165/165 [==============================] - 91s 550ms/step - loss: 0.4555 - accuracy: 0.8216 - val_loss: 1.3551 - val_accuracy: 0.2765
Epoch 6/10
165/165 [==============================] - 91s 551ms/step - loss: 0.4578 - accuracy: 0.8202 - val_loss: 1.3551 - val_accuracy: 0.2765
Epoch 7/10
165/165 [==============================] - 98s 595ms/step - loss: 0.4561 - accuracy: 0.8190 - val_loss: 1.3551 - val_ac

#### En reprocessant les données

In [ ]:
# Définir les paramètres pour la séparation des données 
train_ratio = 0.6 # Pourcentage de données d'entraînement 
validation_ratio = 0.4 # Pourcentage de données de validation

# On reprocesse les jeux de données d'entrainement et de validation
  
# Pas ideal car on applique un image generator sur le jeu de validation

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range = 0.1, # random application of shearing
    zoom_range = 0.1,
    horizontal_flip = False,
    brightness_range = (0.4, 0.6),
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode="nearest",
    validation_split=validation_ratio,
    ) 

test_datagen = ImageDataGenerator(
     preprocessing_function=preprocess_input,
     validation_split=validation_ratio,
     ) 

# Augmenter respectivement les jeu de données d'entrainement et test ?

train_dataset = train_datagen.flow_from_directory(data_dir,batch_size = 64, class_mode="categorical", target_size=(224,224), subset='training', seed=42)

test_dataset = test_datagen.flow_from_directory(data_dir, batch_size = 64, class_mode= "categorical", target_size=(224,224), subset='validation', seed=42)

learning_rate = 0.001 #

# On recrée le modèle de zéro car sinon les poids s ajustent au fur et à mesure

base_model = ResNet50(weights='imagenet')
x = base_model.output
x = Flatten()(x)
x = Dense(60, activation='relu')(x) 
predictions = Dense(4, activation='softmax')(x) 

model = Model(inputs=base_model.input, outputs=predictions)

# on degele les dix dernieres couches:
for layer in model.layers[:165]:
   layer.trainable = False
for layer in model.layers[165:]:
   layer.trainable = True

#Créer une instance de l'optimiseur Adam avec le learning rate personnalisé  
optimizer = Adam(learning_rate=learning_rate) 

# Compiler le modèle avec l'optimiseur personnalisé 
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])

# train the model on the new data for a few epochs
model.fit(train_dataset, # use augmented images for train 
            steps_per_epoch=train_dataset.n // batch_size,
            validation_data = test_dataset, # use initial images for test
            epochs = 2,
            verbose=True)

Found 12701 images belonging to 4 classes.
Found 8464 images belonging to 4 classes.
Epoch 1/2
153/198 [======================>.......] - ETA: 32s - loss: 1.0145 - accuracy: 0.6863

attention les précisions sont élevées ici car je pense que j'ai réentrainé le modèle plusieurs fois : multiplication du nombre d epoch à reprendre .

In [ ]:
# On essaye sans data augmentation
# Définir les paramètres pour la séparation des données 
train_ratio = 0.6 # Pourcentage de données d'entraînement 
validation_ratio = 0.4 # Pourcentage de données de validation

# On reprocesse les jeux de données d'entrainement et de validation avec moins d augmentation de données
  

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range = 0.1, # random application of shearing
    zoom_range = 0.1,
    fill_mode="nearest",
    validation_split=validation_ratio,
    ) 

# test_datagen = ImageDataGenerator(
#     preprocessing_function=preprocess_input,
#     validation_split=validation_ratio,
#     ) 

# Augmenter respectivement les jeu de données d'entrainement

train_dataset = train_datagen.flow_from_directory(data_dir,batch_size = 64, class_mode="categorical", target_size=(224,224), subset='training', seed=42)

test_dataset = train_datagen.flow_from_directory(data_dir, batch_size = 64, class_mode= "categorical", target_size=(224,224), subset='validation', seed=42)

learning_rate = 0.001 #

# On recrée le modèle

base_model = ResNet50(weights='imagenet')
x = base_model.output
x = Flatten()(x)
x = Dense(60, activation='relu')(x) 
predictions = Dense(4, activation='softmax')(x) 

model = Model(inputs=base_model.input, outputs=predictions)

# on degele les dix dernieres couches:
for layer in model.layers[:165]:
   layer.trainable = False
for layer in model.layers[165:]:
   layer.trainable = True


#Créer une instance de l'optimiseur Adam avec le learning rate personnalisé  
optimizer = Adam(learning_rate=learning_rate) 

# Compiler le modèle avec l'optimiseur personnalisé 
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])

# train the model on the new data for a few epochs
model.fit(train_dataset, # use augmented images for train 
            steps_per_epoch=train_dataset.n // batch_size,
            validation_data = test_dataset, # use augmented images for test
            epochs = 10,
            verbose=True)
